In [1]:
import xgboost as xgb  #Libreria de algoriitmo de boosting, sive para hacer la prediccion
from sklearn.metrics import classification_report, confusion_matrix 
import pandas as pd               #Manejo de Datos
import numpy as np                #Calculo numerico
import seaborn as sns             #Libreria de Visualizaciones
import matplotlib.pyplot as plt   #Libreria de Visualizaciones

C:\Users\Tomas\AppData\Local\Continuum\anaconda3\lib\site-packages\dask\dataframe\utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
data1 = pd.read_excel("./Data/salida-sit-1-18022020.xls" ,encode="utf-8", header=1)
data2 = pd.read_excel("./Data/salida-sit-2-21022020.xls",encode="utf-8" , header=1)
data3 = pd.read_excel("./Data/salida-sit-3-26022020.xls" ,encode="utf-8", header=1)

In [3]:
data1= data1.assign(Target=1)
data2= data2.assign(Target=2)
data3= data3.assign(Target=3)

In [4]:
data = pd.concat([data1, data2, data3], axis=0).reset_index()
data.tail()

,index,ID,DNI,CUIT,Nombre completo,Fecha de nacimiento,Clase,Edad,Sexo,Comportamiento 1,...,Monto 4,Banco 5,Sit 5,Periodo 5,Monto 5,Banco 6,Sit 6,Periodo 6,Monto 6,Target
83811,19171,19172,27895095,27278950951,PEREIRA NATALIA SOLEDAD,1980-10-27,1980.0,39.0,F,Emprendimiento La Luisina SRL || Periodo: 11/2...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
83812,19172,19173,16658912,27166589121,CASTELLI JAUREGUI MARIA HELENA,1964-02-02,1964.0,56.0,F,MACRO || Periodo: 12/2019 - Sit: 2000 - Monto:...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
83813,19173,19174,33259345,20332593456,MATUS ALEXIS OSVALDO,1987-05-27,1987.0,32.0,M,ElectrÃ³nica Megatone SA || Periodo: 12/2019 -...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
83814,19174,19175,33354157,20333541573,GODOY RODOLFO EMANUEL,1987-08-22,1987.0,32.0,M,DE GALICIA Y BUENOS AIRES || Periodo: 12/2019 ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
83815,19175,19176,33211146,23332111469,CORDOBA RICARDO EMILIO,1987-09-22,1987.0,32.0,M,DE LA PAMPA SOCIEDAD DE ECONOMA M || Periodo: ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3


In [5]:
data.dtypes

index                    int64
ID                       int64
DNI                      int64
CUIT                     int64
Nombre completo         object
Fecha de nacimiento     object
Clase                  float64
Edad                   float64
Sexo                    object
Comportamiento 1        object
Comportamiento 2        object
Comportamiento 3        object
Comportamiento 4        object
Comportamiento 5        object
Comportamiento 6        object
Comportamiento 7        object
Comportamiento 8        object
Comportamiento 9        object
Comportamiento 10       object
Banco 1                 object
Sit 1                    int64
Periodo 1               object
Monto 1                float64
Banco 2                 object
Sit 2                  float64
Periodo 2               object
Monto 2                float64
Banco 3                 object
Sit 3                  float64
Periodo 3               object
Monto 3                float64
Banco 4                 object
Sit 4   

In [6]:
data[data.duplicated(keep=False)]

,index,ID,DNI,CUIT,Nombre completo,Fecha de nacimiento,Clase,Edad,Sexo,Comportamiento 1,...,Monto 4,Banco 5,Sit 5,Periodo 5,Monto 5,Banco 6,Sit 6,Periodo 6,Monto 6,Target


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83816 entries, 0 to 83815
Data columns (total 44 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   index                83816 non-null  int64  
 1   ID                   83816 non-null  int64  
 2   DNI                  83816 non-null  int64  
 3   CUIT                 83816 non-null  int64  
 4   Nombre completo      83816 non-null  object 
 5   Fecha de nacimiento  83303 non-null  object 
 6   Clase                83758 non-null  float64
 7   Edad                 83299 non-null  float64
 8   Sexo                 83796 non-null  object 
 9   Comportamiento 1     83573 non-null  object 
 10  Comportamiento 2     64050 non-null  object 
 11  Comportamiento 3     44408 non-null  object 
 12  Comportamiento 4     27823 non-null  object 
 13  Comportamiento 5     16131 non-null  object 
 14  Comportamiento 6     8862 non-null   object 
 15  Comportamiento 7     4746 non-null  

In [8]:
nullColumns = []    #Lista que contendrá las columnas con valores Nulos
for column in data.columns:
    if data[column].isnull().sum() > 0:
        print(column, data[column].isnull().sum())
        nullColumns.append(column)

Fecha de nacimiento 513
Clase 58
Edad 517
Sexo 20
Comportamiento 1 243
Comportamiento 2 19766
Comportamiento 3 39408
Comportamiento 4 55993
Comportamiento 5 67685
Comportamiento 6 74954
Comportamiento 7 79070
Comportamiento 8 81280
Comportamiento 9 82446
Comportamiento 10 83058
Monto 1 5
Banco 2 28367
Sit 2 28367
Periodo 2 28367
Monto 2 28368
Banco 3 53454
Sit 3 53454
Periodo 3 53454
Monto 3 53454
Banco 4 69417
Sit 4 69417
Periodo 4 69417
Monto 4 69417
Banco 5 77778
Sit 5 77778
Periodo 5 77778
Monto 5 77780
Banco 6 81862
Sit 6 81862
Periodo 6 81862
Monto 6 81862


In [9]:
to_remove = (["ID","index",'Clase','DNI','CUIT','Nombre completo','Fecha de nacimiento'] + [x for x in data.columns if 'Comportamiento ' in x]+ [x for x in data.columns if 'Periodo ' in x])

In [10]:
bancos = ["Banco 1","Banco 2","Banco 3","Banco 4","Banco 5","Banco 6"]

In [11]:
data.drop(columns = to_remove, inplace=True)

In [12]:
data["Edad"].fillna(data["Edad"].mode()[0], inplace=True)


In [13]:
data["Edad"] =data["Edad"].astype(int)

In [14]:
for banco in bancos:
    data[banco].fillna("desconocido", inplace=True)


In [15]:
data["Sexo"].fillna("desconocido", inplace=True)


In [16]:
columnas_menosbancos = ['Edad','Sexo',
 'Sit 1',
 'Monto 1',
 'Sit 2',
 'Monto 2',
 'Sit 3',
 'Monto 3',
 'Sit 4',
 'Monto 4',
 'Sit 5',
 'Monto 5',
 'Sit 6',
 'Monto 6',
 'Target']

In [17]:
for columna in columnas_menosbancos:
    data[columna].fillna(0, inplace=True)

In [18]:
data

,Edad,Sexo,Banco 1,Sit 1,Monto 1,Banco 2,Sit 2,Monto 2,Banco 3,Sit 3,...,Banco 4,Sit 4,Monto 4,Banco 5,Sit 5,Monto 5,Banco 6,Sit 6,Monto 6,Target
0,74,F,"1Â° de Julio Cooperativa de Vivienda, CrÃ©dito...",1,2000.0,ElectrÃ³nica Megatone SA,1.0,2000.0,MACRO,1.0,...,desconocido,0.0,0.0,desconocido,0.0,0.0,desconocido,0.0,0.0,1
1,58,M,A MUT CLUB A ARGENTINO,1,12000.0,MUNICIPAL DE ROSARIO,1.0,80000.0,NUEVO BANCO DE SANTA FE,1.0,...,desconocido,0.0,0.0,desconocido,0.0,0.0,desconocido,0.0,0.0,1
2,58,F,A MUT CLUB A ARGENTINO,1,14000.0,NUEVO BANCO DE SANTA FE,1.0,48000.0,desconocido,0.0,...,desconocido,0.0,0.0,desconocido,0.0,0.0,desconocido,0.0,0.0,1
3,62,M,A MUT CLUB A ARGENTINO,1,16000.0,desconocido,0.0,0.0,desconocido,0.0,...,desconocido,0.0,0.0,desconocido,0.0,0.0,desconocido,0.0,0.0,1
4,67,M,A MUT CLUB A ARGENTINO,1,16000.0,HIPOTECARIO,1.0,16000.0,NUEVO BANCO DE SANTA FE,1.0,...,RIO DE LA PLATA / SANTANDER RIO,1.0,33000.0,TARJETAS DEL MAR,1.0,8000.0,desconocido,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83811,39,F,Emprendimiento La Luisina SRL,3,6000.0,desconocido,0.0,0.0,desconocido,0.0,...,desconocido,0.0,0.0,desconocido,0.0,0.0,desconocido,0.0,0.0,3
83812,56,F,MACRO,3,2000.0,desconocido,0.0,0.0,desconocido,0.0,...,desconocido,0.0,0.0,desconocido,0.0,0.0,desconocido,0.0,0.0,3
83813,32,M,ElectrÃ³nica Megatone SA,3,30000.0,desconocido,0.0,0.0,desconocido,0.0,...,desconocido,0.0,0.0,desconocido,0.0,0.0,desconocido,0.0,0.0,3
83814,32,M,DE GALICIA Y BUENOS AIRES,3,21000.0,desconocido,0.0,0.0,desconocido,0.0,...,desconocido,0.0,0.0,desconocido,0.0,0.0,desconocido,0.0,0.0,3


In [19]:
data.shape

(83816, 21)

In [20]:
#MONTO PONDEARADO

In [21]:
data["monto_ponderado"]= data["Monto 1"] * data["Sit 1"]+data["Monto 2"] * data["Sit 2"]+data["Monto 3"] * data["Sit 3"]+data["Monto 4"] * data["Sit 4"]+data["Monto 5"] * data["Sit 5"]+data["Monto 6"] * data["Sit 6"]

In [22]:
data['sit1_1'] = (data['Sit 1']!=0).astype(int)
data['sit1_2'] = (data['Sit 2']!=0).astype(int)
data['sit1_3'] = (data['Sit 3']!=0).astype(int)
data['sit1_4'] = (data['Sit 4']!=0).astype(int)
data['sit1_5'] = (data['Sit 5']!=0).astype(int)
data['sit1_6'] = (data['Sit 6']!=0).astype(int)
data['cantidad_de_bancos'] = data[["sit1_1", "sit1_2", "sit1_3", "sit1_4", "sit1_5", "sit1_6"]].sum(axis=1)

In [23]:
data['sit1_1'] = (data['Sit 1']>2).astype(int)
data['sit1_2'] = (data['Sit 2']>2).astype(int)
data['sit1_3'] = (data['Sit 3']>2).astype(int)
data['sit1_4'] = (data['Sit 4']>2).astype(int)
data['sit1_5'] = (data['Sit 5']>2).astype(int)
data['sit1_6'] = (data['Sit 6']>2).astype(int)
data['cantidad_creditos_morosos'] = data[["sit1_1", "sit1_2", "sit1_3", "sit1_4", "sit1_5", "sit1_6"]].sum(axis=1)

In [36]:
#monto MOROSo

data.loc[data['Sit 1'] >2, 'sit1_1'] = data["Monto 1"]
data.loc[data['Sit 2'] >2, 'sit1_2'] = data["Monto 2"]
data.loc[data['Sit 3'] >2, 'sit1_3'] = data["Monto 3"]
data.loc[data['Sit 4'] >2, 'sit1_4'] = data["Monto 4"]
data.loc[data['Sit 5'] >2, 'sit1_5'] = data["Monto 5"]
data.loc[data['Sit 6'] >2, 'sit1_6'] = data["Monto 6"]
data['suma_morosa'] = data[["sit1_1", "sit1_2", "sit1_3", "sit1_4", "sit1_5", "sit1_6"]].sum(axis=1)

In [40]:
#promedio de situaciones
data['prom_sit(target)']= data[["Sit 1", "Sit 2", "Sit 3", "Sit 4", "Sit 5", "Sit 6"]].sum(axis=1) / data['cantidad_de_bancos']

In [41]:
data

,Edad,Sexo,Banco 1,Sit 1,Monto 1,Banco 2,Sit 2,Monto 2,Banco 3,Sit 3,...,sit_3,sit_4,sit_5,credito moroso,sit_6,monto_total,monto_prom_porbanco,sit1_11,suma morosa,prom_sit(target)
0,74,F,"1Â° de Julio Cooperativa de Vivienda, CrÃ©dito...",1,2000.0,ElectrÃ³nica Megatone SA,1.0,2000.0,MACRO,1.0,...,0,0,0,0,0,104000.0,34666.666667,NaN,0.0,1.0
1,58,M,A MUT CLUB A ARGENTINO,1,12000.0,MUNICIPAL DE ROSARIO,1.0,80000.0,NUEVO BANCO DE SANTA FE,1.0,...,0,0,0,0,0,221000.0,73666.666667,NaN,0.0,1.0
2,58,F,A MUT CLUB A ARGENTINO,1,14000.0,NUEVO BANCO DE SANTA FE,1.0,48000.0,desconocido,0.0,...,0,0,0,0,0,62000.0,31000.000000,NaN,0.0,1.0
3,62,M,A MUT CLUB A ARGENTINO,1,16000.0,desconocido,0.0,0.0,desconocido,0.0,...,0,0,0,0,0,16000.0,16000.000000,NaN,0.0,1.0
4,67,M,A MUT CLUB A ARGENTINO,1,16000.0,HIPOTECARIO,1.0,16000.0,NUEVO BANCO DE SANTA FE,1.0,...,0,0,0,0,0,86000.0,17200.000000,NaN,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83811,39,F,Emprendimiento La Luisina SRL,3,6000.0,desconocido,0.0,0.0,desconocido,0.0,...,0,0,0,1,0,6000.0,6000.000000,6000.0,6000.0,3.0
83812,56,F,MACRO,3,2000.0,desconocido,0.0,0.0,desconocido,0.0,...,0,0,0,1,0,2000.0,2000.000000,2000.0,2000.0,3.0
83813,32,M,ElectrÃ³nica Megatone SA,3,30000.0,desconocido,0.0,0.0,desconocido,0.0,...,0,0,0,1,0,30000.0,30000.000000,30000.0,30000.0,3.0
83814,32,M,DE GALICIA Y BUENOS AIRES,3,21000.0,desconocido,0.0,0.0,desconocido,0.0,...,0,0,0,1,0,21000.0,21000.000000,21000.0,21000.0,3.0


In [28]:
#monto total
data["monto_total"] =  data["Monto 1"]+data["Monto 2"]  +data["Monto 3"] +data["Monto 4"]+data["Monto 5"] +data["Monto 6"]

In [29]:
# MONTO PROMEDIO POR BANCO
data["monto_prom_porbanco"] =data["monto_total"] / data['cantidad_de_bancos']

In [146]:
data["Edad"].fillna(data["Edad"].mode()[0], inplace=True)
data["Edad"].unique()

array([ 74,  58,  62,  67,  33,  37,  40,  45,  38,  81,  44,  73,  25,
        34,  41,  52,  49,  36,  65,  51,  47,  39,  71,  66,  50,  57,
        64,  61,  48,  80,  79,  32,  55,  46,  78,  54,  53,  75,  68,
        70,  72,  43,  56,  30,  29,  60,  28,  42,  35,  31,  63,  87,
        69,  90,  77,   4,  26,  16,   3,  10, 106,  21,  18,  76,  59,
        20,  23,  83,  93,  88,  85,  14,  92,  94,  12,  15,   9, 107,
        27,  95,  19,  13,  82, 104,  17, 119,  24,   5,  22,  11,  89,
        86,  84,   8, 108, -29,   6,   7, -26, -28, -23,   2, 219, 220,
       -14,  99, 110, -27, -24,   1, 101, -20,  98, -30, 120, 111,  91,
        97, 115,  96, 118, 105, -15,  -9, -16, 140, 114, 100, -17, 113,
       -22])

In [147]:
data.columns.tolist()

['Edad',
 'Sexo',
 'Banco 1',
 'Sit 1',
 'Monto 1',
 'Banco 2',
 'Sit 2',
 'Monto 2',
 'Banco 3',
 'Sit 3',
 'Monto 3',
 'Banco 4',
 'Sit 4',
 'Monto 4',
 'Banco 5',
 'Sit 5',
 'Monto 5',
 'Banco 6',
 'Sit 6',
 'Monto 6',
 'Target',
 'monto_ponderado',
 'sit1_1',
 'sit1_2',
 'sit1_3',
 'sit1_4',
 'sit1_5',
 'sit1_6',
 'cantidad_de_bancos',
 'cantidad_creditos_morosos']

In [148]:
cat= ["Sexo"]
data["Sexo"].shape


(83816,)

In [149]:
data["Sexo"].unique()


array(['F', 'M', 'C', 'desconocido'], dtype=object)

In [150]:
data["Target"].head(121711)

0        1
1        1
2        1
3        1
4        1
        ..
83811    3
83812    3
83813    3
83814    3
83815    3
Name: Target, Length: 83816, dtype: int64

In [151]:
OH_cols=pd.get_dummies(data["Sexo"])

In [152]:
from sklearn.preprocessing import LabelEncoder

# Apply label encoder to each column with categorical data
for banco in bancos:
    label_encoder = LabelEncoder()
    data[banco] = label_encoder.fit_transform(data[banco])




In [153]:
data.reindex
data

,Edad,Sexo,Banco 1,Sit 1,Monto 1,Banco 2,Sit 2,Monto 2,Banco 3,Sit 3,...,Target,monto_ponderado,sit1_1,sit1_2,sit1_3,sit1_4,sit1_5,sit1_6,cantidad_de_bancos,cantidad_creditos_morosos
0,74,F,0,1,2000.0,132,1.0,2000.0,158,1.0,...,1,106000.0,0,0,0,0,0,0,3,0
1,58,M,1,1,12000.0,190,1.0,80000.0,177,1.0,...,1,233000.0,0,0,0,0,0,0,3,0
2,58,F,1,1,14000.0,206,1.0,48000.0,242,0.0,...,1,76000.0,0,0,0,0,0,0,2,0
3,62,M,1,1,16000.0,268,0.0,0.0,242,0.0,...,1,32000.0,0,0,0,0,0,0,1,0
4,67,M,1,1,16000.0,168,1.0,16000.0,177,1.0,...,1,102000.0,0,0,0,0,0,0,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83811,39,F,150,3,6000.0,268,0.0,0.0,242,0.0,...,3,36000.0,1,0,0,0,0,0,1,1
83812,56,F,201,3,2000.0,268,0.0,0.0,242,0.0,...,3,12000.0,1,0,0,0,0,0,1,1
83813,32,M,149,3,30000.0,268,0.0,0.0,242,0.0,...,3,180000.0,1,0,0,0,0,0,1,1
83814,32,M,130,3,21000.0,268,0.0,0.0,242,0.0,...,3,126000.0,1,0,0,0,0,0,1,1


In [154]:
data = data.drop("Sexo", axis = 1)
data = data.drop("Edad", axis = 1)

In [155]:
data_j = data.join(OH_cols)

In [156]:
data_j["Target"].unique()

array([1, 2, 3], dtype=int64)

In [170]:
X = data_j.drop(["Target","cantidad_creditos_morosos"], axis=1)
y = data_j["Target"]

In [171]:
from sklearn.model_selection import train_test_split
X_train_m1, X_test_m1, y_train_m1, y_test_m1 = train_test_split( X,y, test_size=0.3, random_state=7)

In [173]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score


classifier=DecisionTreeClassifier(criterion="entropy",random_state=0)
classifier.fit(X_train_m1,y_train_m1)
y_pred_m1=classifier.predict(X_test_m1)
acc=accuracy_score(y_test_m1, y_pred_m1)
print(acc)


0.9998011533107973


In [174]:
import eli5
from eli5.sklearn import PermutationImportance
from sklearn.svm import SVC

# ... load data

perm = PermutationImportance(classifier).fit(X_test_m1, y_test_m1)
eli5.show_weights(perm)

Weight,Feature
0.3092 ± 0.0033,x1
0.1599 ± 0.0018,x4
0.1258 ± 0.0017,x19
0.0738 ± 0.0024,x7
0.0334 ± 0.0005,x10
0.0167 ± 0.0008,x13
0.0157 ± 0.0007,x21
0.0082 ± 0.0004,x16
0.0027 ± 0.0002,x22
0.0014 ± 0.0005,x23


In [175]:
eli5.show_weights(perm, feature_names = X_train_m1.columns.tolist())

Weight,Feature
0.3092 ± 0.0033,Sit 1
0.1599 ± 0.0018,Sit 2
0.1258 ± 0.0017,sit1_1
0.0738 ± 0.0024,Sit 3
0.0334 ± 0.0005,Sit 4
0.0167 ± 0.0008,Sit 5
0.0157 ± 0.0007,sit1_3
0.0082 ± 0.0004,Sit 6
0.0027 ± 0.0002,sit1_4
0.0014 ± 0.0005,sit1_5


In [160]:
from xgboost import XGBClassifier

xgbc = XGBClassifier()
xgbc.fit(X_train_m1, y_train_m1)
xgbc_pred =xgbc.predict(X_test_m1)
print(classification_report(y_test_m1,xgbc_pred))
xgbc.score(X_test_m1, y_test_m1)

              precision    recall  f1-score   support

           1       1.00      1.00      1.00     15010
           2       1.00      1.00      1.00      4370
           3       1.00      1.00      1.00      5765

    accuracy                           1.00     25145
   macro avg       1.00      1.00      1.00     25145
weighted avg       1.00      1.00      1.00     25145



0.9999602306621594

In [166]:
from sklearn.inspection import permutation_importance
r = permutation_importance(xgbc, X_test_m1, y_test_m1,
                            n_repeats=30,
                            random_state=0)

for i in r.importances_mean.argsort()[::-1]:
        
    if r.importances_mean[i] - 2 * r.importances_std[i] > 0:
        
        print(f"{diabetes.feature_names[i]:<8}"
              f"{r.importances_mean[i]:.3f}"
              f" +/- {r.importances_std[i]:.3f}")

NameError: name 'diabetes' is not defined

In [164]:
y.value_counts()

1    50000
3    19176
2    14640
Name: Target, dtype: int64